In [ ]:
import pandas as pd
from pymongo import MongoClient
import datetime

def _connect_mongo(db,username, password):
    """ A util for making a connection to mongo """

    mongo_uri = 'mongo.dsi.ic.ac.uk:27017'
    conn = MongoClient(mongo_uri,tz_aware=True)
    database = conn[db]
    database.authenticate(username, password)
    return database


def get_collections(db, username=None, password=None):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(db, username, password)
    all_collections = db.list_collection_names()
    print("Length : " ,len(all_collections))
    return all_collections

def get_data(db, username=None, password=None, no_id=True, collection=None, query={}):
    db = _connect_mongo(db, username, password)
    cursor = db[collection].find(query).limit(3)
    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))
    return df
    
def get_erc_data(db, username=None, password=None, no_id=True, collection=None, query={}):
    db = _connect_mongo(db, username, password)

    cursor = db[collection].find({'tx_time' : { '$gte': '2018-01-01T01:00:00.000+00:00' }})
    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))
    df["time"] = pd.to_datetime(pd.to_datetime(df['tx_time']).dt.strftime("%Y-%m-%d %H:%M:%S"))

    return df

In [ ]:
def get_erc_data_for_collections(collections):
    final = []
    for collection in collections:
        print(collection)
        dff = get_erc_data('ERC20Analysis','pratha','ChangeMe42', collection=collection)
        dff['token'] = collection
        final.append(dff)
    return final

In [ ]:
final = get_erc_data_for_collections(['OMG'])
dff = pd.concat(final)
dff.count()

In [ ]:
dff.to_csv('OMG_data.csv')